In [1]:
# import libraries
import pandas as pd
import numpy as np
import nbformat as nbf
import matplotlib.pyplot as plt
import plotly_express as px

In [2]:
# read vehicles_us.csv into a DataFrame

df = pd.read_csv('../csv_files/vehicles_us.csv', parse_dates=['date_posted'])

# display the first 5 rows of the DataFrame
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [3]:
# get the median model_year by model
median_model_year = df.groupby('model')['model_year'].median()

# get the median model_year of the entire dataset
median_model_year = median_model_year.median()

# fill missing values in model_year with the median model_year for the model that is missing the value
df['model_year'] = df.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median()))

# fill remaining missing values in model_year with the median model year of the entire dataset
df['model_year'] = df['model_year'].fillna(median_model_year)

# convert model_year to integer
df['model_year'] = df['model_year'].astype(int)



In [4]:
# get the median value of cylinders by type
median_cylinders = df.groupby('type')['cylinders'].median()

# fill missing values in cylinders with the median cylinders value for the type that is missing the value
df['cylinders'] = df.groupby('type')['cylinders'].transform(lambda x: x.fillna(x.median()))


In [5]:
# convert is_4wd col to bool
df['is_4wd'] = df['is_4wd'].astype(bool)

In [6]:
# fill  missing paint_color values with 'unknown'
df['paint_color'] = df['paint_color'].fillna('unknown')

In [7]:
# Calculate the median odometer value of the entire dataset
median_odometer = df['odometer'].median()

# Calculate the median odometer value for each model_year
median_odometer_by_year = df.groupby('model_year')['odometer'].median()

# Fill missing odometer values with the median odometer value of the corresponding model_year
df['odometer'] = df.groupby('model_year')['odometer'].transform(lambda x: x.fillna(x.median()))

# Fill remaining missing odometer values with the median odometer value of the entire dataset
df['odometer'] = df['odometer'].fillna(median_odometer)

/Users/ncullings/Documents/GitHub/repositories/car-sales-ads-dashboard/.venv/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [8]:
# Check for fractional parts for the odometer column
df['has_fraction'] = df['odometer'].apply(lambda x: x % 1 != 0)

# Evaluate whether data loss would occur upon converting odometer values to integers
data_loss_occurs = df['has_fraction'].any()
# print("Data loss occurs:", data_loss_occurs)

# Drop the has_fraction column
df = df.drop('has_fraction', axis=1)

In [9]:
# Check for fractional parts for the odometer column
df['has_fraction'] = df['cylinders'].apply(lambda x: x % 1 != 0)

# Evaluate whether data loss would occur upon converting odometer values to integers
data_loss_occurs = df['has_fraction'].any()
print("Data loss occurs:", data_loss_occurs)

# Drop the has_fraction column
df = df.drop('has_fraction', axis=1)

# Convert the cylinders column to int
df['cylinders'] = df['cylinders'].astype(int)

Data loss occurs: False


In [10]:
# Split the 'model' column into two separate columns 'make' and 'model'
df[['make', 'model']] = df['model'].str.split(' ', n=1, expand=True)


In [11]:
# confirm that there are no duplicates
print('Number of duplicates:', df.duplicated().sum())

Number of duplicates: 0


In [12]:
# confirm data types are as expected and that there are no missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  int64         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  int64         
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  bool          
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  make          51525 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 5.2+ MB


In [13]:
df.describe()

,price,model_year,cylinders,odometer,date_posted,days_listed
count,51525.000000,51525.000000,51525.000000,51525.000000,51525,51525.00000
mean,12132.464920,2009.793557,6.130810,115199.265609,2018-10-25 01:57:46.270742528,39.55476
min,1.000000,1908.000000,3.000000,0.000000,2018-05-01 00:00:00,0.00000
25%,5000.000000,2007.000000,4.000000,73500.000000,2018-07-29 00:00:00,19.00000
50%,9000.000000,2011.000000,6.000000,114072.000000,2018-10-25 00:00:00,33.00000
75%,16839.000000,2014.000000,8.000000,152827.000000,2019-01-21 00:00:00,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,2019-04-19 00:00:00,271.00000
std,10040.803015,6.099381,1.658414,62082.376858,NaN,28.20427


In [14]:
# clean-up implicit model duplicates

# replace 'sd' with 'super duty' in the model column where the make is 'ford'
df.loc[df['make'] == 'ford', 'model'] = df.loc[df['make'] == 'ford', 'model'].str.replace('sd', 'super duty')

# remove '-' from the model column where the make is 'ford'
df.loc[df['make'] == 'ford', 'model'] = df.loc[df['make'] == 'ford', 'model'].str.replace('-', '')

In [15]:
# confirm that there are no remaining implicit duplicates for ford makes

# filter df for ford makes
filtered_df = df[df['make'].str.contains('ford')]

# get unique combinations of make and model then print them
unique_combinations = filtered_df[['make', 'model']].drop_duplicates()
print(unique_combinations)


     make                   model
1    ford                    f150
26   ford               fusion se
48   ford                   focus
54   ford  f150 supercrew cab xlt
63   ford                 mustang
79   ford         f250 super duty
88   ford     mustang gt coupe 2d
103  ford                explorer
109  ford         f350 super duty
127  ford                    edge
131  ford                    f250
148  ford              expedition
154  ford                  taurus
166  ford                  ranger
194  ford                  escape
266  ford                  fusion
429  ford                focus se
713  ford                    f350
849  ford               econoline


In [16]:
# create a scatter plot of price by model year using plotly.express
fig = px.scatter(df, 
                 x='model_year', 
                 y='price', 
                 color='condition',
                 title='Price by Model Year',
                 labels={'model_year': 'Model Year', 'price': 'Price', 'condition': 'Condition'}
                 )

# update the x-axis range to start from 1950 and end at the maximum model year
#fig.update_xaxes(range=[1950,max(df['model_year'])])

# update the y-axis range to start from 0 and end at 200,000
#fig.update_yaxes(range=[0, 200000])

# show the plot
fig.show()


In [17]:
# Create a grouped_df of price grouped by condition
grouped_df = df.groupby('condition')['price'].agg(['mean', 'median', 'var','std'])

# sort the grouped_df by condition_categories and print the result as integers
sorted_df = grouped_df.reindex(condition_categories)
print(sorted_df.astype(int))

NameError: name 'condition_categories' is not defined

In [ ]:
# Plot a histogram of model_year
fig = px.histogram(df, 
                   x='model_year', 
                   title='Histogram of Model Year',
                   labels={'model_year': 'Model Year', 'count': 'Count'}
                  )

# update the x-axis range to start from 1950 and end at the maximum model year
# fig.update_xaxes(range=[1980,max(df['model_year'])])

# update the y-axis title
fig.update_yaxes(title_text='Number of Listings')


# show the plot
fig.show()




In [ ]:
# Make a list of the ordered condition categories
condition_categories = ['salvage', 'fair', 'good', 'excellent', 'like new', 'new']

# Plot a histogram of condition
fig = px.histogram(df,
                   x='condition',
                   title='Histogram of Vehicle Condition',
                   labels={'condition': 'Condition', 'count': 'Count'},
                   category_orders={'condition': condition_categories}
                   )

# Show the plot
fig.show()

In [ ]:
# Plot a scatter plot of odometer by price
fig = px.scatter(df,
                 x='odometer',
                 y='price',
                 labels={'odometer': 'Odometer (Miles)', 'price': 'Price'},
                 )

# Show the plot
fig.show()

In [ ]:
corr_matrix = df[['odometer','price']].corr()
print(f"Correlation between the odometer value and vehicle price: {corr_matrix.loc['odometer','price']:.2f}")

# Calculate the mean, median, variance, and standard deviation of the odometer and price columns
agg_stats = df[['odometer','price']].agg(['mean', 'median', 'var','std'])

# Round the values to 0 decimal places and print the result
agg_stats = agg_stats.map(lambda x: f'{x:.0f}')
print("\n", agg_stats)


In [ ]:
# export the df to a new csv file named 'vehicles_us_cleaned.csv' without the index and save it in the current directory
df.to_csv('vehicles_us_cleaned.csv', index=False)